# TypeScript Debounce関数 実装

## 1. 問題の分析

### 競技プログラミング視点での分析
- **実行速度**: debounce自体は遅延が目的なので、呼び出しのオーバーヘッドを最小化
- **メモリ使用量**: タイマーID1つと最新の引数のみ保持（O(1)空間）
- **アルゴリズム**: シンプルなタイマー管理で十分

### 業務開発視点での分析
- **型安全性**: 引数の型を正確に保持し、実行時エラーを防止
- **保守性**: クロージャによる状態管理で明確な責務分離
- **メモリリーク防止**: タイマーの適切なクリアが必須
- **エラーハンドリング**: 不正な`t`値の検証

### TypeScript特有の考慮点
- **型推論**: `ReturnType<typeof setTimeout>`でタイマーIDの型を自動推論
- **クロージャの型安全性**: 外部変数の型を厳密に管理
- **ジェネリクス**: より汎用的な実装も可能だが、LeetCode形式に従う

## 2. アルゴリズムアプローチ比較

|アプローチ|時間計算量|空間計算量|TS実装コスト|型安全性|可読性|備考|
|---------|---------|---------|-----------|-------|------|-----|
|タイマー管理|O(1)|O(1)|低|高|高|標準的なdebounce実装|
|キュー方式|O(n)|O(n)|高|中|低|過剰設計、不要|

## 3. 選択したアルゴリズムと理由

**選択したアプローチ**: タイマー管理方式

**理由**:
- **計算量的な優位性**: 各呼び出しO(1)、メモリもO(1)で最適
- **TypeScript環境での型安全性**: クロージャで型情報を完全に保持
- **保守性・可読性**: 実装が直感的で、debounceの動作が明確

**TypeScript特有の最適化ポイント**:
- `ReturnType<typeof setTimeout>`による型推論活用
- strict nullチェックによる安全なタイマー管理
- クロージャによる状態カプセル化

## 4. 実装コード

### LeetCode Performance
- Runtime: 48 ms (Beats 71.15%)
- Memory: 54.18 MB (Beats 95.14%)

In [ ]:
// Type definition for function signature
type F = (...args: number[]) => void

In [ ]:
/**
 * 関数の実行をデバウンスする（遅延実行＆キャンセル機能付き）
 * @param fn - デバウンス対象の関数
 * @param t - 遅延時間（ミリ秒）
 * @returns デバウンスされた関数
 * @complexity Time: O(1) per call, Space: O(1)
 */
function debounce(fn: F, t: number): F {
    // タイマーIDを保持するクロージャ変数（型安全）
    let timeoutId: ReturnType<typeof setTimeout> | null = null;
    
    return function(...args: number[]): void {
        // 既存のタイマーがあればキャンセル
        if (timeoutId !== null) {
            clearTimeout(timeoutId);
        }
        
        // 新しいタイマーをセット（t ミリ秒後に fn を実行）
        timeoutId = setTimeout(() => {
            fn(...args);
        }, t);
    };
}

In [ ]:
// Usage example
const log = debounce(console.log, 100);
log('Hello'); // cancelled
log('Hello'); // cancelled
log('Hello'); // Logged at t=100ms

## 5. 実装の詳細説明

### コア機能

1. **タイマー管理**
   - `timeoutId`変数で現在のタイマーを追跡
   - `null`チェックで型安全性を確保

2. **キャンセルメカニズム**
   - 新しい呼び出し時に既存タイマーを`clearTimeout`でクリア
   - これにより前の実行がキャンセルされる

3. **遅延実行**
   - `setTimeout`で`t`ミリ秒後に元の関数を実行
   - 引数は最新の呼び出し時のものを使用

### TypeScript型安全性のポイント

```typescript
// ✅ 型安全な実装
let timeoutId: ReturnType<typeof setTimeout> | null = null;
// - ReturnType: setTimeoutの戻り値型を自動推論
// - | null: 初期状態とクリア後の状態を表現
// - strict null checkで安全性確保

// ✅ 引数の型保持
return function(...args: number[]): void {
    // argsの型が明示的にnumber[]として保持される
}
```

### 動作例の詳細

**Example 1**: `t = 50ms`
```
50ms: dlog(1) → タイマーセット（100msに実行予定）
75ms: dlog(2) → 前のタイマークリア、新タイマーセット（125msに実行）
125ms: fn(2)実行
```

**Example 2**: `t = 20ms`
```
50ms: dlog(1) → タイマーセット（70msに実行予定）
70ms: fn(1)実行
100ms: dlog(2) → タイマーセット（120msに実行）
120ms: fn(2)実行
```

## TypeScript固有の最適化観点

### 型安全性の活用

1. **コンパイル時エラー防止**
   - `timeoutId`の`null`チェックで未初期化エラーを防止
   - strict modeでのnull安全性確保

2. **型推論による開発効率**
   - `ReturnType<typeof setTimeout>`で環境依存の型を自動取得
   - Node.js/ブラウザ両方で動作

3. **クロージャの型安全性**
   - 外部変数の型が明確で、スコープ管理が安全

### パフォーマンス特性

- **時間計算量**: O(1) - 各呼び出しは定数時間
- **空間計算量**: O(1) - タイマーID1つのみ保持
- **メモリリーク**: なし - タイマーは適切にクリアされる

### エッジケース対応

- `t = 0`: `setTimeout(fn, 0)`は次のイベントループティックで実行される（同期的ではない）。連続呼び出しでは最後の呼び出しのみが実行されるdebounce動作は維持される
- 連続呼び出し: 最後の呼び出しのみが実行される
- 引数なし: 正常に動作（`...args`が空配列）